# Recommender Systems project
## SLIM with structure

In [1]:
import sys
sys.path.append('.')

Import all the Recommenders to be evaluated

In [2]:
from SLIM_Elastic_Net_Structured.SLIM_Elastic_Net_Structured import SLIM_Elastic_Net_Structured
from KNN.item_knn_CF import ItemKNNCFRecommender
from KNN.item_knn_CBF import ItemKNNCBFRecommender
from SLIM_RMSE.SLIM_RMSE import SLIM_RMSE

### Load the datasets on which we want to compare the algorithms

### TheMoviesDataset

In [3]:
 from scipy import sparse as sps

TMD_URM = sps.load_npz("./data/TheMoviesDataset/URM_all.npz")
TMD_ICM = sps.load_npz("./data/TheMoviesDataset/ICM_all.npz")



### TheMoviesDataset 10% and 50% features

In [4]:
import numpy as np

features_frequency = TMD_ICM.sum(axis=0)
features_arg = np.asarray(np.argsort(features_frequency.squeeze())).reshape(-1)[::-1]
features_arg10 = features_arg[:len(features_arg)//10]
features_arg50 = features_arg[:len(features_arg)//2]

TMD_ICM10 = TMD_ICM[:,features_arg10]
TMD_ICM50 = TMD_ICM[:,features_arg50]

print(TMD_ICM10.shape)
print(TMD_ICM50.shape)

(45433, 4910)
(45433, 24551)


In [5]:
print("TMD_ICM10 Sparcity: {}".format(1 - TMD_ICM10.nnz/(TMD_ICM10.shape[0]*TMD_ICM10.shape[1])))
print("TMD_ICM50 Sparcity: {}".format(1 - TMD_ICM50.nnz/(TMD_ICM50.shape[0]*TMD_ICM50.shape[1])))

TMD_ICM10 Sparcity: 0.998379794548074
TMD_ICM50 Sparcity: 0.9994530078838976


#### Splitting in train and test the URM

In [10]:
from sklearn.model_selection import train_test_split

print(TMD_URM.shape)
print(TMD_ICM.shape)

TMD_URM_train, TMD_URM_test = train_test_split(TMD_URM, test_size=0.33)

(270896, 45433)
(45433, 49102)


In [7]:
print(TMD_URM_train.shape)
print(TMD_URM_test.shape)

(181500, 45433)
(89396, 45433)


## Movielens26

In [4]:
 from scipy import sparse as sps

M26_ICM_GEN  = sps.load_npz("./data/ml-latest/icm26M_genres.npz")
M26_ICM_TAG = sps.load_npz("./data/ml-latest/icm26M_tags.npz")
M26_URM = sps.load_npz("./data/ml-latest/urm26M.npz")

#### Splitting in train and test the URM

In [5]:
from sklearn.model_selection import train_test_split

print(M26_ICM_GEN.shape)
print(M26_ICM_TAG.shape)
print(M26_URM.shape)

M26_URM_train, M26_URM_test = train_test_split(M26_URM, test_size=0.33)

(45843, 20)
(45843, 53508)
(270896, 45843)


In [6]:
print(M26_URM_train.shape)
print(M26_URM_test.shape)

(181500, 45843)
(89396, 45843)


## Start testing

### TheMovieDataSet

### KNN collaborative filtering

In [21]:
KNNrecommender = ItemKNNCFRecommender(TMD_URM_train)
KNNrecommender.fit()

Similarity column 10000 ( 22 % ), 241.79 column/sec, elapsed time 0.69 min
Similarity column 20000 ( 44 % ), 364.78 column/sec, elapsed time 0.91 min
Similarity column 30000 ( 66 % ), 487.80 column/sec, elapsed time 1.03 min
Similarity column 40000 ( 88 % ), 600.95 column/sec, elapsed time 1.11 min
Similarity column 45433 ( 100 % ), 657.82 column/sec, elapsed time 1.15 min


In [26]:
results_KNNCFR = KNNrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_KNNCFR

{'AUC': 0.16328739386752908,
 'MRR': 0.18905935960684533,
 'NDCG': 0.018557255206137824,
 'map': 0.09102503054552336,
 'precision': 0.12380611457273409,
 'recall': 0.009259586463904812}

### KNN content base filtering

In [30]:
KNNCBrecommender = ItemKNNCBFRecommender(TMD_ICM, TMD_URM_train)
KNNCBrecommender.fit()

Similarity column 10000 ( 22 % ), 1414.99 column/sec, elapsed time 0.12 min
Similarity column 20000 ( 44 % ), 1456.93 column/sec, elapsed time 0.23 min
Similarity column 30000 ( 66 % ), 1516.19 column/sec, elapsed time 0.33 min
Similarity column 40000 ( 88 % ), 1563.21 column/sec, elapsed time 0.43 min
Similarity column 45433 ( 100 % ), 1567.11 column/sec, elapsed time 0.48 min


In [31]:
results_KNNCBFR = KNNCBrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_KNNCBFR

Processed 10000 ( 11.19% ) in 358.21 seconds. Users per second: 28
Processed 20000 ( 22.37% ) in 715.80 seconds. Users per second: 28
Processed 30000 ( 33.56% ) in 1073.20 seconds. Users per second: 28
Processed 40000 ( 44.75% ) in 1431.92 seconds. Users per second: 28
Processed 50000 ( 55.93% ) in 1768.91 seconds. Users per second: 28
Processed 60000 ( 67.12% ) in 2083.54 seconds. Users per second: 29
Processed 70000 ( 78.31% ) in 2397.96 seconds. Users per second: 29
Processed 80000 ( 89.49% ) in 2714.02 seconds. Users per second: 29
Processed 89392 ( 100.00% ) in 3007.67 seconds. Users per second: 30


{'AUC': 0.10482457612266392,
 'MRR': 0.111912192975588,
 'NDCG': 0.007462012060254801,
 'map': 0.03904454673930252,
 'precision': 0.058389359345799155,
 'recall': 0.0036111568186750448}

### SLIM ElasticNet

In [33]:
SLIMRSErecommender = SLIM_RMSE(TMD_URM_train)
SLIMRSErecommender.fit()

Processed 1344 ( 2.96% ) in 5.00 minutes. Columns per second: 4
Processed 3157 ( 6.95% ) in 10.00 minutes. Columns per second: 5
Processed 4807 ( 10.58% ) in 15.01 minutes. Columns per second: 5
Processed 6422 ( 14.14% ) in 20.01 minutes. Columns per second: 5
Processed 8506 ( 18.72% ) in 25.01 minutes. Columns per second: 6
Processed 9979 ( 21.96% ) in 30.01 minutes. Columns per second: 6
Processed 11935 ( 26.27% ) in 35.01 minutes. Columns per second: 6
Processed 13620 ( 29.98% ) in 40.06 minutes. Columns per second: 6
Processed 15077 ( 33.19% ) in 45.06 minutes. Columns per second: 6
Processed 16253 ( 35.77% ) in 50.15 minutes. Columns per second: 5
Processed 17378 ( 38.25% ) in 55.15 minutes. Columns per second: 5
Processed 18495 ( 40.71% ) in 60.23 minutes. Columns per second: 5
Processed 19106 ( 42.05% ) in 65.24 minutes. Columns per second: 5
Processed 19704 ( 43.37% ) in 70.32 minutes. Columns per second: 5
Processed 20152 ( 44.36% ) in 75.35 minutes. Columns per second: 4
Proc

In [34]:
results_SLIMRSE = SLIMRSErecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_SLIMRSE

Processed 10000 ( 11.19% ) in 10.59 seconds. Users per second: 944
Processed 20000 ( 22.37% ) in 21.10 seconds. Users per second: 948
Processed 30000 ( 33.56% ) in 31.49 seconds. Users per second: 953
Processed 40000 ( 44.75% ) in 41.78 seconds. Users per second: 957
Processed 50000 ( 55.93% ) in 52.24 seconds. Users per second: 957
Processed 60000 ( 67.12% ) in 62.63 seconds. Users per second: 958
Processed 70000 ( 78.31% ) in 72.85 seconds. Users per second: 961
Processed 80000 ( 89.49% ) in 83.16 seconds. Users per second: 962
Processed 89392 ( 100.00% ) in 92.49 seconds. Users per second: 967


{'AUC': 0.22638051450710178,
 'MRR': 0.26451362709235704,
 'NDCG': 0.031131544579922704,
 'map': 0.12724350775665486,
 'precision': 0.16969785106218715,
 'recall': 0.014939901420999723}

## Structured SLIM

In [6]:
strucSLIMrecommender = SLIM_Elastic_Net_Structured(TMD_ICM, TMD_URM_train)
strucSLIMrecommender.fit()

SLIM ElasticNet with structure, beginning of the fit process.

Similarity column 10000 ( 22 % ), 1726.56 column/sec, elapsed time 0.10 min
Similarity column 20000 ( 44 % ), 1795.39 column/sec, elapsed time 0.19 min
Similarity column 30000 ( 66 % ), 1842.97 column/sec, elapsed time 0.27 min
Similarity column 40000 ( 88 % ), 1878.42 column/sec, elapsed time 0.35 min
Similarity column 45433 ( 100 % ), 1890.54 column/sec, elapsed time 0.40 min

Start the learning process:
Processed 1000 (  2 % ), 3.48 column/sec, elapsed time 4.79 min
Processed 2000 (  4 % ), 3.37 column/sec, elapsed time 9.88 min
Processed 3000 (  7 % ), 3.72 column/sec, elapsed time 13.43 min
Processed 4000 (  9 % ), 4.16 column/sec, elapsed time 16.02 min
Processed 5000 ( 11 % ), 4.69 column/sec, elapsed time 17.77 min
Processed 6000 ( 13 % ), 5.25 column/sec, elapsed time 19.06 min
Processed 7000 ( 15 % ), 5.71 column/sec, elapsed time 20.42 min
Processed 8000 ( 18 % ), 6.23 column/sec, elapsed time 21.40 min
Processed

In [7]:
results_strucSLIM = strucSLIMrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_strucSLIM

Processed 10000 ( 11.19% ) in 11.60 seconds. Users per second: 862
Processed 20000 ( 22.37% ) in 21.87 seconds. Users per second: 914
Processed 30000 ( 33.56% ) in 32.44 seconds. Users per second: 925
Processed 40000 ( 44.75% ) in 42.88 seconds. Users per second: 933
Processed 50000 ( 55.94% ) in 53.63 seconds. Users per second: 932
Processed 60000 ( 67.12% ) in 63.96 seconds. Users per second: 938
Processed 70000 ( 78.31% ) in 75.20 seconds. Users per second: 931
Processed 80000 ( 89.50% ) in 86.03 seconds. Users per second: 930
Processed 89390 ( 100.00% ) in 95.28 seconds. Users per second: 938


{'AUC': 0.0712925984345927,
 'MRR': 0.07144455258359018,
 'NDCG': 0.005127760900578022,
 'map': 0.01937692584015973,
 'precision': 0.03291382801400354,
 'recall': 0.00237117981474157}

### TheMoviesDataset 10% of features

### KNN content base filtering

In [8]:
KNNCBrecommender = ItemKNNCBFRecommender(TMD_ICM10, TMD_URM_train)
KNNCBrecommender.fit()

Similarity column 10000 ( 22 % ), 1609.49 column/sec, elapsed time 0.10 min
Similarity column 20000 ( 44 % ), 1648.59 column/sec, elapsed time 0.20 min
Similarity column 30000 ( 66 % ), 1659.78 column/sec, elapsed time 0.30 min
Similarity column 40000 ( 88 % ), 1700.74 column/sec, elapsed time 0.39 min
Similarity column 45433 ( 100 % ), 1727.78 column/sec, elapsed time 0.44 min


In [9]:
results_KNNCBFR = KNNCBrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_KNNCBFR

Processed 10000 ( 11.19% ) in 356.55 seconds. Users per second: 28
Processed 20000 ( 22.37% ) in 704.32 seconds. Users per second: 28
Processed 30000 ( 33.56% ) in 1062.96 seconds. Users per second: 28
Processed 40000 ( 44.75% ) in 1442.66 seconds. Users per second: 28
Processed 50000 ( 55.94% ) in 1796.80 seconds. Users per second: 28
Processed 60000 ( 67.12% ) in 2149.17 seconds. Users per second: 28
Processed 70000 ( 78.31% ) in 2559.40 seconds. Users per second: 27
Processed 80000 ( 89.50% ) in 2943.71 seconds. Users per second: 27
Processed 89389 ( 100.00% ) in 3274.11 seconds. Users per second: 27


{'AUC': 0.08979285527836814,
 'MRR': 0.0954351717194279,
 'NDCG': 0.006027635885299432,
 'map': 0.031691075934418146,
 'precision': 0.04852668083677559,
 'recall': 0.0029754989239676242}

### Structured SLIM

In [10]:
strucSLIMrecommender = SLIM_Elastic_Net_Structured(TMD_ICM50, TMD_URM_train)
strucSLIMrecommender.fit()

SLIM ElasticNet with structure, beginning of the fit process.

Similarity column 10000 ( 22 % ), 1750.18 column/sec, elapsed time 0.10 min
Similarity column 20000 ( 44 % ), 1796.06 column/sec, elapsed time 0.19 min
Similarity column 30000 ( 66 % ), 1834.28 column/sec, elapsed time 0.27 min
Similarity column 40000 ( 88 % ), 1865.76 column/sec, elapsed time 0.36 min
Similarity column 45433 ( 100 % ), 1806.63 column/sec, elapsed time 0.42 min

Start the learning process:
Processed 1000 (  2 % ), 3.42 column/sec, elapsed time 4.88 min
Processed 2000 (  4 % ), 3.35 column/sec, elapsed time 9.94 min
Processed 3000 (  7 % ), 3.68 column/sec, elapsed time 13.58 min
Processed 4000 (  9 % ), 4.10 column/sec, elapsed time 16.25 min
Processed 5000 ( 11 % ), 4.61 column/sec, elapsed time 18.06 min
Processed 6000 ( 13 % ), 5.16 column/sec, elapsed time 19.40 min
Processed 7000 ( 15 % ), 5.61 column/sec, elapsed time 20.78 min
Processed 8000 ( 18 % ), 6.12 column/sec, elapsed time 21.79 min
Processed

In [11]:
results_strucSLIM = strucSLIMrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_strucSLIM

Processed 10000 ( 11.19% ) in 9.06 seconds. Users per second: 1103
Processed 20000 ( 22.37% ) in 18.16 seconds. Users per second: 1102
Processed 30000 ( 33.56% ) in 27.22 seconds. Users per second: 1102
Processed 40000 ( 44.75% ) in 36.29 seconds. Users per second: 1102
Processed 50000 ( 55.94% ) in 45.39 seconds. Users per second: 1102
Processed 60000 ( 67.12% ) in 54.48 seconds. Users per second: 1101
Processed 70000 ( 78.31% ) in 63.63 seconds. Users per second: 1100
Processed 80000 ( 89.50% ) in 72.72 seconds. Users per second: 1100
Processed 89389 ( 100.00% ) in 81.27 seconds. Users per second: 1100


{'AUC': 0.07471752992504764,
 'MRR': 0.0748273483238209,
 'NDCG': 0.00525156182861393,
 'map': 0.020370107270263175,
 'precision': 0.03421635529701036,
 'recall': 0.0023486699132642124}

### TheMoviesDataset 50% of features

### KNN content base filtering

In [12]:
KNNCBrecommender = ItemKNNCBFRecommender(TMD_ICM50, TMD_URM_train)
KNNCBrecommender.fit()

Similarity column 10000 ( 22 % ), 1300.55 column/sec, elapsed time 0.13 min
Similarity column 20000 ( 44 % ), 1323.75 column/sec, elapsed time 0.25 min
Similarity column 30000 ( 66 % ), 1482.12 column/sec, elapsed time 0.34 min
Similarity column 40000 ( 88 % ), 1583.66 column/sec, elapsed time 0.42 min
Similarity column 45433 ( 100 % ), 1622.28 column/sec, elapsed time 0.47 min


In [13]:
results_KNNCBFR = KNNCBrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_KNNCBFR

Processed 10000 ( 11.19% ) in 353.83 seconds. Users per second: 28
Processed 20000 ( 22.37% ) in 706.81 seconds. Users per second: 28
Processed 30000 ( 33.56% ) in 1060.15 seconds. Users per second: 28
Processed 40000 ( 44.75% ) in 1415.48 seconds. Users per second: 28
Processed 50000 ( 55.94% ) in 1769.44 seconds. Users per second: 28
Processed 60000 ( 67.12% ) in 2123.73 seconds. Users per second: 28
Processed 70000 ( 78.31% ) in 2483.21 seconds. Users per second: 28
Processed 80000 ( 89.50% ) in 2838.10 seconds. Users per second: 28
Processed 89389 ( 100.00% ) in 3168.37 seconds. Users per second: 28


{'AUC': 0.10088097102584181,
 'MRR': 0.10794440093970242,
 'NDCG': 0.007064354060380539,
 'map': 0.03703610893587213,
 'precision': 0.055847410224847426,
 'recall': 0.0034313059550974195}

### Structured SLIM

In [11]:
strucSLIMrecommender = SLIM_Elastic_Net_Structured(TMD_ICM50, TMD_URM_train)
strucSLIMrecommender.fit()

SLIM ElasticNet with structure, beginning of the fit process.

Similarity column 10000 ( 22 % ), 1568.11 column/sec, elapsed time 0.11 min
Similarity column 20000 ( 44 % ), 1660.62 column/sec, elapsed time 0.20 min
Similarity column 30000 ( 66 % ), 1700.51 column/sec, elapsed time 0.29 min
Similarity column 40000 ( 88 % ), 1733.08 column/sec, elapsed time 0.38 min
Similarity column 45433 ( 100 % ), 1751.19 column/sec, elapsed time 0.43 min

Start the learning process:
Processed 1000 (  2 % ), 3.39 column/sec, elapsed time 4.92 min
Processed 2000 (  4 % ), 3.28 column/sec, elapsed time 10.16 min
Processed 3000 (  7 % ), 3.60 column/sec, elapsed time 13.87 min
Processed 4000 (  9 % ), 4.02 column/sec, elapsed time 16.59 min
Processed 5000 ( 11 % ), 4.51 column/sec, elapsed time 18.46 min
Processed 6000 ( 13 % ), 5.04 column/sec, elapsed time 19.84 min
Processed 7000 ( 15 % ), 5.48 column/sec, elapsed time 21.29 min
Processed 8000 ( 18 % ), 5.97 column/sec, elapsed time 22.33 min
Processe

In [12]:
results_strucSLIM = strucSLIMrecommender.evaluateRecommendations(TMD_URM_test, at=5, exclude_seen=True)
results_strucSLIM

Processed 10000 ( 11.19% ) in 9.34 seconds. Users per second: 1071
Processed 20000 ( 22.37% ) in 18.63 seconds. Users per second: 1074
Processed 30000 ( 33.56% ) in 27.94 seconds. Users per second: 1074
Processed 40000 ( 44.75% ) in 37.24 seconds. Users per second: 1074
Processed 50000 ( 55.94% ) in 46.58 seconds. Users per second: 1073
Processed 60000 ( 67.12% ) in 55.90 seconds. Users per second: 1073
Processed 70000 ( 78.31% ) in 65.24 seconds. Users per second: 1073
Processed 80000 ( 89.50% ) in 74.54 seconds. Users per second: 1073
Processed 89390 ( 100.00% ) in 83.27 seconds. Users per second: 1073


{'AUC': 0.07454609524448776,
 'MRR': 0.07474540688286659,
 'NDCG': 0.0051414034822329195,
 'map': 0.020058438644704992,
 'precision': 0.03371927822711188,
 'recall': 0.002295724200388743}

### Movielens26 GENRES

### KNN collaborative filtering

In [7]:
KNNrecommender = ItemKNNCFRecommender(M26_URM_train)
KNNrecommender.fit()

results_KNNCFR = KNNrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_KNNCFR

Similarity column 10000 ( 22 % ), 254.26 column/sec, elapsed time 0.66 min
Similarity column 20000 ( 44 % ), 376.53 column/sec, elapsed time 0.89 min
Similarity column 30000 ( 65 % ), 477.80 column/sec, elapsed time 1.05 min
Similarity column 40000 ( 87 % ), 579.29 column/sec, elapsed time 1.15 min
Similarity column 45843 ( 100 % ), 628.76 column/sec, elapsed time 1.22 min
Processed 10000 ( 11.19% ) in 316.24 seconds. Users per second: 32
Processed 20000 ( 22.37% ) in 621.54 seconds. Users per second: 32
Processed 30000 ( 33.56% ) in 896.19 seconds. Users per second: 33
Processed 40000 ( 44.75% ) in 1170.96 seconds. Users per second: 34
Processed 50000 ( 55.93% ) in 1442.64 seconds. Users per second: 35
Processed 60000 ( 67.12% ) in 1717.20 seconds. Users per second: 35
Processed 70000 ( 78.30% ) in 1985.89 seconds. Users per second: 35
Processed 80000 ( 89.49% ) in 2253.71 seconds. Users per second: 35
Processed 89395 ( 100.00% ) in 2506.51 seconds. Users per second: 36


{'AUC': 0.1653122063627012,
 'MRR': 0.19139018151447576,
 'NDCG': 0.018977900038729845,
 'map': 0.09108725657623755,
 'precision': 0.12431652422927689,
 'recall': 0.00957347798587911}

### KNN content base filtering

In [8]:
KNNCBrecommender = ItemKNNCBFRecommender(M26_ICM_GEN, M26_URM_train)
KNNCBrecommender.fit()

results_KNNCBFR = KNNCBrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_KNNCBFR

Similarity column 10000 ( 22 % ), 1513.69 column/sec, elapsed time 0.11 min
Similarity column 20000 ( 44 % ), 1531.81 column/sec, elapsed time 0.22 min
Similarity column 30000 ( 65 % ), 1571.23 column/sec, elapsed time 0.32 min
Similarity column 40000 ( 87 % ), 1612.17 column/sec, elapsed time 0.41 min
Similarity column 45843 ( 100 % ), 1624.36 column/sec, elapsed time 0.47 min
Processed 10000 ( 11.19% ) in 263.77 seconds. Users per second: 38
Processed 20000 ( 22.37% ) in 528.09 seconds. Users per second: 38
Processed 30000 ( 33.56% ) in 791.15 seconds. Users per second: 38
Processed 40000 ( 44.75% ) in 1054.84 seconds. Users per second: 38
Processed 50000 ( 55.93% ) in 1318.51 seconds. Users per second: 38
Processed 60000 ( 67.12% ) in 1581.81 seconds. Users per second: 38
Processed 70000 ( 78.30% ) in 1845.02 seconds. Users per second: 38
Processed 80000 ( 89.49% ) in 2108.05 seconds. Users per second: 38
Processed 89395 ( 100.00% ) in 2355.10 seconds. Users per second: 38


{'AUC': 0.024136985994899086,
 'MRR': 0.023511678374871336,
 'NDCG': 0.0011077027865926744,
 'map': 0.005952615329545095,
 'precision': 0.011776813280237004,
 'recall': 0.0006147582326954524}

### SLIM ElasticNet

In [9]:
SLIMRSErecommender = SLIM_RMSE(M26_URM_train)
SLIMRSErecommender.fit()

results_SLIMRSE = SLIMRSErecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_SLIMRSE

Processed 1381 ( 3.01% ) in 5.00 minutes. Columns per second: 5
Processed 3218 ( 7.02% ) in 10.00 minutes. Columns per second: 5
Processed 5352 ( 11.67% ) in 15.00 minutes. Columns per second: 6
Processed 7408 ( 16.16% ) in 20.00 minutes. Columns per second: 6
Processed 9244 ( 20.16% ) in 25.01 minutes. Columns per second: 6
Processed 11153 ( 24.33% ) in 30.01 minutes. Columns per second: 6
Processed 13173 ( 28.74% ) in 35.01 minutes. Columns per second: 6
Processed 14626 ( 31.90% ) in 40.04 minutes. Columns per second: 6
Processed 15633 ( 34.10% ) in 45.11 minutes. Columns per second: 6
Processed 16722 ( 36.48% ) in 50.13 minutes. Columns per second: 6
Processed 17793 ( 38.81% ) in 55.13 minutes. Columns per second: 5
Processed 18842 ( 41.10% ) in 60.25 minutes. Columns per second: 5
Processed 19529 ( 42.60% ) in 65.28 minutes. Columns per second: 5
Processed 20043 ( 43.72% ) in 70.33 minutes. Columns per second: 5
Processed 20551 ( 44.83% ) in 75.37 minutes. Columns per second: 5
Pro

{'AUC': 0.22716340775873528,
 'MRR': 0.2636167539188936,
 'NDCG': 0.03100365924008567,
 'map': 0.126326774501488,
 'precision': 0.16803212671709916,
 'recall': 0.014697972614191545}

## Structured SLIM

In [10]:
strucSLIMrecommender = SLIM_Elastic_Net_Structured(M26_ICM_GEN, M26_URM_train)
strucSLIMrecommender.fit()

results_strucSLIM = strucSLIMrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_strucSLIM

SLIM ElasticNet with structure, beginning of the fit process.

Similarity column 10000 ( 22 % ), 1445.71 column/sec, elapsed time 0.12 min
Similarity column 20000 ( 44 % ), 1455.81 column/sec, elapsed time 0.23 min
Similarity column 30000 ( 65 % ), 1496.21 column/sec, elapsed time 0.33 min
Similarity column 40000 ( 87 % ), 1540.93 column/sec, elapsed time 0.43 min
Similarity column 45843 ( 100 % ), 1553.19 column/sec, elapsed time 0.49 min

Start the learning process:
Processed 1000 (  2 % ), 3.70 column/sec, elapsed time 4.50 min
Processed 2000 (  4 % ), 3.60 column/sec, elapsed time 9.25 min
Processed 3000 (  7 % ), 3.94 column/sec, elapsed time 12.69 min
Processed 4000 (  9 % ), 4.35 column/sec, elapsed time 15.33 min
Processed 5000 ( 11 % ), 4.88 column/sec, elapsed time 17.09 min
Processed 6000 ( 13 % ), 5.43 column/sec, elapsed time 18.40 min
Processed 7000 ( 15 % ), 5.92 column/sec, elapsed time 19.70 min
Processed 8000 ( 17 % ), 6.41 column/sec, elapsed time 20.81 min
Processed

{'AUC': 0.0792913926051874,
 'MRR': 0.07872350739033451,
 'NDCG': 0.005022348592007627,
 'map': 0.020681754341481277,
 'precision': 0.033979148955207615,
 'recall': 0.00219208177663743}

### Movielens26 TAGS

### KNN collaborative filtering

In [11]:
KNNrecommender = ItemKNNCFRecommender(M26_URM_train)
KNNrecommender.fit()

results_KNNCFR = KNNrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_KNNCFR

Similarity column 10000 ( 22 % ), 283.85 column/sec, elapsed time 0.59 min
Similarity column 20000 ( 44 % ), 417.64 column/sec, elapsed time 0.80 min
Similarity column 30000 ( 65 % ), 548.11 column/sec, elapsed time 0.91 min
Similarity column 40000 ( 87 % ), 665.85 column/sec, elapsed time 1.00 min
Similarity column 45843 ( 100 % ), 727.90 column/sec, elapsed time 1.05 min
Processed 10000 ( 11.19% ) in 270.62 seconds. Users per second: 37
Processed 20000 ( 22.37% ) in 541.27 seconds. Users per second: 37
Processed 30000 ( 33.56% ) in 812.69 seconds. Users per second: 37
Processed 40000 ( 44.75% ) in 1083.40 seconds. Users per second: 37
Processed 50000 ( 55.93% ) in 1354.52 seconds. Users per second: 37
Processed 60000 ( 67.12% ) in 1627.08 seconds. Users per second: 37
Processed 70000 ( 78.30% ) in 1898.36 seconds. Users per second: 37
Processed 80000 ( 89.49% ) in 2169.11 seconds. Users per second: 37
Processed 89395 ( 100.00% ) in 2425.66 seconds. Users per second: 37


{'AUC': 0.1653122063627012,
 'MRR': 0.19139018151447576,
 'NDCG': 0.018977900038729845,
 'map': 0.09108725657623755,
 'precision': 0.12431652422927689,
 'recall': 0.00957347798587911}

### KNN content base filtering

In [12]:
KNNCBrecommender = ItemKNNCBFRecommender(M26_ICM_TAG, M26_URM_train)
KNNCBrecommender.fit()

results_KNNCBFR = KNNCBrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_KNNCBFR

Similarity column 10000 ( 22 % ), 2431.70 column/sec, elapsed time 0.07 min
Similarity column 20000 ( 44 % ), 2598.37 column/sec, elapsed time 0.13 min
Similarity column 30000 ( 65 % ), 2740.82 column/sec, elapsed time 0.18 min
Similarity column 40000 ( 87 % ), 2844.59 column/sec, elapsed time 0.23 min
Similarity column 45843 ( 100 % ), 2901.64 column/sec, elapsed time 0.26 min
Processed 10000 ( 11.19% ) in 180.71 seconds. Users per second: 55
Processed 20000 ( 22.37% ) in 360.27 seconds. Users per second: 56
Processed 30000 ( 33.56% ) in 539.82 seconds. Users per second: 56
Processed 40000 ( 44.75% ) in 720.28 seconds. Users per second: 56
Processed 50000 ( 55.93% ) in 900.16 seconds. Users per second: 56
Processed 60000 ( 67.12% ) in 1080.21 seconds. Users per second: 56
Processed 70000 ( 78.30% ) in 1262.17 seconds. Users per second: 55
Processed 80000 ( 89.49% ) in 1440.81 seconds. Users per second: 56
Processed 89395 ( 100.00% ) in 1608.85 seconds. Users per second: 56


{'AUC': 0.09306531239279914,
 'MRR': 0.09721575909436314,
 'NDCG': 0.006260217303837109,
 'map': 0.03198887534117678,
 'precision': 0.04987471475232986,
 'recall': 0.003100330259478841}

### SLIM ElasticNet

In [13]:
SLIMRSErecommender = SLIM_RMSE(M26_URM_train)
SLIMRSErecommender.fit()

results_SLIMRSE = SLIMRSErecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_SLIMRSE

Processed 1345 ( 2.93% ) in 5.00 minutes. Columns per second: 4
Processed 3124 ( 6.81% ) in 10.00 minutes. Columns per second: 5
Processed 5172 ( 11.28% ) in 15.00 minutes. Columns per second: 6
Processed 7178 ( 15.66% ) in 20.00 minutes. Columns per second: 6
Processed 9025 ( 19.69% ) in 25.00 minutes. Columns per second: 6
Processed 10578 ( 23.07% ) in 30.01 minutes. Columns per second: 6
Processed 12488 ( 27.24% ) in 35.11 minutes. Columns per second: 6
Processed 14206 ( 30.99% ) in 40.11 minutes. Columns per second: 6
Processed 15409 ( 33.61% ) in 45.11 minutes. Columns per second: 6
Processed 16245 ( 35.44% ) in 50.11 minutes. Columns per second: 5
Processed 17504 ( 38.18% ) in 55.17 minutes. Columns per second: 5
Processed 18638 ( 40.66% ) in 60.17 minutes. Columns per second: 5
Processed 19273 ( 42.04% ) in 65.17 minutes. Columns per second: 5
Processed 19874 ( 43.35% ) in 70.20 minutes. Columns per second: 5
Processed 20308 ( 44.30% ) in 75.20 minutes. Columns per second: 5
Pro

{'AUC': 0.22714569630259254,
 'MRR': 0.2636120930093823,
 'NDCG': 0.03100516643268517,
 'map': 0.12632975748357522,
 'precision': 0.16804331289992616,
 'recall': 0.01469823350972618}

## Structured SLIM

In [14]:
strucSLIMrecommender = SLIM_Elastic_Net_Structured(M26_ICM_TAG, M26_URM_train)
strucSLIMrecommender.fit()

results_strucSLIM = strucSLIMrecommender.evaluateRecommendations(M26_URM_test, at=5, exclude_seen=True)
results_strucSLIM

SLIM ElasticNet with structure, beginning of the fit process.

Similarity column 10000 ( 22 % ), 2475.88 column/sec, elapsed time 0.07 min
Similarity column 20000 ( 44 % ), 2641.61 column/sec, elapsed time 0.13 min
Similarity column 30000 ( 65 % ), 2776.96 column/sec, elapsed time 0.18 min
Similarity column 40000 ( 87 % ), 2877.94 column/sec, elapsed time 0.23 min
Similarity column 45843 ( 100 % ), 2920.00 column/sec, elapsed time 0.26 min

Start the learning process:
Processed 1000 (  2 % ), 3.16 column/sec, elapsed time 5.27 min
Processed 2000 (  4 % ), 3.06 column/sec, elapsed time 10.90 min
Processed 3000 (  7 % ), 3.35 column/sec, elapsed time 14.90 min
Processed 4000 (  9 % ), 3.70 column/sec, elapsed time 18.03 min
Processed 5000 ( 11 % ), 4.16 column/sec, elapsed time 20.05 min
Processed 6000 ( 13 % ), 4.64 column/sec, elapsed time 21.56 min
Processed 7000 ( 15 % ), 5.06 column/sec, elapsed time 23.06 min
Processed 8000 ( 17 % ), 5.49 column/sec, elapsed time 24.29 min
Processe

{'AUC': 0.032017651796500984,
 'MRR': 0.030920660133935844,
 'NDCG': 0.0021933634099461337,
 'map': 0.007511139573732084,
 'precision': 0.01480826882634708,
 'recall': 0.0011481825420549612}